In [1]:
import tensorflow as tf
import seaborn as sns
import numpy as np
import keras

Using TensorFlow backend.


In [2]:
import albumentations as A

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten,BatchNormalization,Lambda
from tensorflow.keras.layers import Conv2D, MaxPooling2D


In [5]:
from ImageDataAugmentor.image_data_augmentor import *

In [6]:
from albumentations import (
    HorizontalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, RandomBrightnessContrast, IAAPiecewiseAffine,
    IAASharpen, IAAEmboss, Flip, OneOf, Compose,ToFloat,RandomGamma
)

In [7]:
def augment(aug,image):
    image = aug(image=image)['image']
    return image

def augment_2d(inputs,p=.5):
    return augment(Compose([
        RandomRotate90(p=p),
        Flip(p=p),
        Transpose(p=p),
        OneOf([
            IAAAdditiveGaussianNoise(),
            GaussNoise(),
        ], p=0.2),
        OneOf([
            MotionBlur(p=.2),
            MedianBlur(blur_limit=3, p=0.1),
            Blur(blur_limit=3, p=0.1),
        ], p=0.2),
        ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=45, p=0.2),
        OneOf([
            OpticalDistortion(p=0.3),
            GridDistortion(p=.1),
            IAAPiecewiseAffine(p=0.3),
        ], p=0.2),
        OneOf([
            CLAHE(clip_limit=2),
            IAASharpen(),
            IAAEmboss(),
            RandomBrightnessContrast(),            
        ], p=0.3),
        HueSaturationValue(p=0.3),
        ToFLoat(max_value=255)
    ], p=p),inputs)
    

In [74]:
import cv2
p = 0.5
import albumentations
AUGMENTATIONS_TRAIN = Compose([
   RandomRotate90(p=p),
        Flip(p=p),
        Transpose(p=p),
        OneOf([
            IAAAdditiveGaussianNoise(),
            GaussNoise(),
        ], p=0.2),
        OneOf([
            MotionBlur(p=.2),
            MedianBlur(blur_limit=3, p=0.1),
            Blur(blur_limit=3, p=0.1),
        ], p=0.2),
        ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=45, p=0.2),
        OneOf([
            OpticalDistortion(p=0.3),
            GridDistortion(p=.1),
            IAAPiecewiseAffine(p=0.3),
        ], p=0.2),
        OneOf([
            CLAHE(clip_limit=2),
            IAASharpen(),
            IAAEmboss(),
            RandomBrightnessContrast(),            
        ], p=0.3),
        HueSaturationValue(p=0.3)])

AUGMENTATIONS = albumentations.Compose([
    albumentations.Transpose(p=0.5),
    albumentations.Flip(p=0.5),
    albumentations.OneOf([
        albumentations.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3),
        albumentations.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1)
    ],p=1),
    albumentations.GaussianBlur(p=0.05),
    albumentations.HueSaturationValue(p=0.5),
    albumentations.RGBShift(p=0.5),
])

AUGMENTATIONS_TEST = Compose([
    ToFloat(max_value=255)
])

In [9]:
import os

In [75]:
train_datagen = ImageDataAugmentor(
    rescale=1./255,
    augment=AUGMENTATIONS,
    preprocess_input=None
)
test_datagen = ImageDataAugmentor(rescale=1./255)

In [76]:
train_generator = train_datagen.flow_from_directory(
'images',
target_size=(256,256),
    batch_size=32,
class_mode='categorical')

Found 194996 images belonging to 39 classes.


In [77]:
def find_id_dir(dirname):
    return [os.path.join(dirname,fname) for fname in sorted(os.listdir(dirname))]

In [78]:
train_generator.image_shape

(256, 256, 3)

In [79]:
def get_model():
    model = tf.keras.models.Sequential()
    model.add(Conv2D(3,6,5,input_shape=train_generator.image_shape))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(2,2))
    model.add(Conv2D(6,16,5))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Dense(120))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.3))
    model.add(Dense(85))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.3))
    model.add(Dense(39,activation='softmax'))
    return model

In [80]:
def get_opt():
    return tf.keras.optimizers.Adam()


In [81]:
import tensorboard
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, TensorBoard

In [82]:
from tensorboard import notebook
notebook.list()

Known TensorBoard instances:
  - port 6006: logdir logs (started 2:43:49 ago; pid 3380)


In [83]:
# stop learining as metric on validatopn stop increasing
early_stopping = EarlyStopping(patience=10, verbose=1, mode = 'auto') 

# tensorboard for monitoring logs
tensorboard = TensorBoard(log_dir='./logs', histogram_freq=1,
                          write_graph=True, write_images=True)
callbacks = [tensorboard]


In [ ]:
model = get_model()
opt = get_opt()
model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])
model.fit_generator(train_generator,epochs=10,
                    steps_per_epoch=len(train_generator),callbacks=callbacks)

W0105 15:41:54.735765 139665238980416 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']


Train for 6094 steps
Epoch 1/10
6094/6094 [==============================] - 1082s 178ms/step - loss: 3.5274 - accuracy: 0.0601
Epoch 2/10
6094/6094 [==============================] - 1107s 182ms/step - loss: 3.4116 - accuracy: 0.0843
Epoch 3/10
6094/6094 [==============================] - 1133s 186ms/step - loss: 3.3762 - accuracy: 0.0927
Epoch 4/10
6094/6094 [==============================] - 1200s 197ms/step - loss: 3.3578 - accuracy: 0.0963
Epoch 5/10
6094/6094 [==============================] - 1140s 187ms/step - loss: 3.3465 - accuracy: 0.0998
Epoch 6/10
6094/6094 [==============================] - 1185s 194ms/step - loss: 3.3382 - accuracy: 0.1019
Epoch 7/10
4483/6094 [=====================>........] - ETA: 5:37 - loss: 3.3323 - accuracy: 0.1037

In [39]:
tf.__version__

'2.1.0-rc2'

In [45]:
import cv2

In [46]:
import PIL

In [51]:
from cv2 import colorChange

In [72]:
base = '/home/arturx/ml/mushrooms classification/'
for dirname in find_id_dir('images'):
    print(dirname+str(len(find_id_dir(dirname))))
    for fname in find_id_dir(dirname):
        fname = base+fname
        try:
            img = cv2.cv2.imread(fname, cv2.IMREAD_COLOR)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        except:
            print(fname)

images/amanita+muscaria5000
images/amanita+pantherina5000
images/amanita+phalloides5000
images/amanita+verna5000
images/armillaria+gallica5000
images/armillaria+mellea5000
images/boletus+betulicola5000
images/boletus+edulis5000
images/boletus+pinicola5000
images/boletus+pinophilus5000
images/boletus+satanas5000
images/cantharellus+cibarius5000
images/coprinellus+micaceus5000
images/fomes+fomentarius5000
images/fomitopsis+pinicola5000
images/galerina+marginata5000
images/ganoderma+pfeifferi5000
images/hypholoma+fasciculare5000
images/lactarius+delicious5000
images/lactarius+deterrimus5000
images/leccinum+melaneum5000
images/leccinum+scabrum5000
images/leccinum+variicolor5000
images/leccinum+versipelle5000
images/leccinum+vulpinum5000
images/mycena+galericulata5000
images/pluteus+cervinus5000
images/psathyrella+candolleana5000
images/russula+aeruginea5000
images/russula+claroflava5000
images/russula+vesca5000
images/russula+xerampelina5000
images/suillus+flavidus5000
images/suillus+granu